<img src="images/kiksmeisedwengougent.png" alt="Banner" width="1100">

<div>
    <font color=#690027 markdown="1">   
        <h1>HOOGTE BOMEN EN AFMETINGEN STOMATA IN HET AMAZONEWOUD</h1>
    </font>
</div>

<div class="alert alert-box alert-success">
Onderzoekers uit Brazilië onderzochten met lineaire regressie of er een verband is tussen het aantal huidmondjes op bladeren in de kruin van een boom en de hoogte van de boom. In de vorige notebook ging je met hun data aan de slag en paste je er zelf lineaire regressie op toe. <br>
    In deze notebook voeg je enkele elementen uit machinaal leren toe aan die werkwijze. In machinaal leren reserveren de computerwetenschappers een deel van de data om de kwaliteit van het model na te gaan. Net als zij zal je in deze notebook werken met trainingdata en testdata.
</div>

Paleoklimatologen hebben aangetoond dat er een verband is tussen het aantal en de grootte van stomata op bladeren en het CO<sub>2</sub>-gehalte in de atmosfeer toen deze planten groeiden.<br>
Vandaag de dag wordt er wereldwijd door wetenschappers onderzoek gedaan naar de huidmondjes op bladeren van nu. <br> Bij sommige planten ontdekte men verschillen in de stomata van bladeren ontsproten in de lente tegenover die in de zomer. Bij andere planten stelde men verschillen vast tussen bladeren in de kruin van een plant en de beschaduwde bladeren onderaan in de plant.<br>
Vast staat dat het aantal huidmondjes en hun grootte onderhevig zijn aan omgevingsfactoren.<br> <br>
De onderzoekers Camargo en Marenco uit Brazilië vroegen zich het volgende af:<br>
**Is er een verband tussen het aantal huidmondjes op bladeren in de kruin van een boom en de hoogte van de boom?**<br>
Om dit te onderzoeken gebruikten ze data verzameld in het Amazonewoud [1].

<div class="alert alert-box alert-warning">
Je zal merken dat het begin van deze notebook idientiek verloopt aan de vorige notebook <em>Stomata Hoogte Bomen</em>. Vanaf puntje 4. komt daar echter verandering in. 
</div>

### Nodige modules importeren

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np

from scipy.optimize import curve_fit    # voor regressie
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

<div>
    <font color=#690027 markdown="1">   
        <h2>1. Inlezen van de data</h2>
    </font>
</div>

In [ ]:
amazone = pd.read_csv("data/amazone.csv")  

<div>
    <font color=#690027 markdown="1">   
        <h2>2. Tonen van de ingelezen data</h2> 
    </font>
</div>

In [ ]:
# dataset weergeven in tabel
amazone

<div>
    <font color=#690027 markdown="1">   
        <h2>3. De lineaire samenhang tussen de data nagaan via de correlatiecoëfficiënt</h2> 
    </font>
</div>

<div class="alert alert-block alert-warning">
Meer uitleg over de correlatiecoëfficiënt vind je in de notebook 'Standaardiseren'.
</div>

Beschouw de twee kenmerken die je nodig hebt.

Je zal hier werken met de data opgeslagen in `x1` en `x3` (`x1` refereert aan de stomatale dichtheden en `x3` aan de hoogtes van de bomen). Bij de grafische voorstelling komt `x3` op de y-as en `x1` op de x-as.

In [ ]:
x1 = amazone["stomatale dichtheid"] 
x3 = amazone["hoogte boom"]    

Bepaal de correlatiecoëfficiënt R voor de hoogte van de boom en de dichtheid van de stomata. Is er een sterke, matige of zwakke lineaire samenhang tussen deze kenmerken?

In [ ]:
# in hoeverre is er lineair verband tussen hoogte van boom en dichtheid van stomata? 
np.corrcoef(x3, x1)[0,1]

Er is een zwakke lineaire samenhang tussen de hoogte van de boom en de stomatale dichtheid!

<div>
    <font color=#690027 markdown="1">   
        <h2>4. Regressielijn voor het verband tussen stomatale dichtheid en de hoogte van de boom</h2> 
    </font>
</div>

In machinaal leren reserveren de computerwetenschappers een deel van de data om de kwaliteit van het model na te gaan. 

<div class="alert alert-box alert-info">
Binnen machinaal leren gaat men als volgt te werk: de data worden opgesplitst in trainingdata en testdata.<br> <em>De trainingdata worden gebruikt om een wiskundig model op te stellen. <br>Met de testdata wordt nagegaan of het model goed presteert op nieuwe data.</em><br>
Om te kijken hoe goed het model presteert, wordt de gemiddelde kwadratische afwijking berekend.
</div>

<div>
    <font color=#690027 markdown="1">   
        <h3>4.1 Trainingdata en testdata</h3> 
    </font>
</div>

Je zal werken met **trainingdata en testdata**.

In [ ]:
# oorspronkelijke data
x1 = np.array(x1) 
x3 = np.array(x3) 

# trainingdata
x_train = x3[0:29]
y_train = x1[0:29]

# testdata
x_test = x3[29:]
y_test = x1[29:]

**De trainingdata en testdata worden los van elkaar gestandaardiseerd.**

<div>
    <font color=#690027 markdown="1">   
        <h3>4.2 Standaardiseren</h3> 
    </font>
</div>

De training- en testdata worden als volgt **gestandaardiseerd**: van elke gegeven uit de trainingdata wordt het gemiddelde van de trainingdata afgetrokken en vervolgens wordt het resultaat gedeeld door de standaardafwijking van de trainingdata. <br> 
De volledige dataset wordt op dezelfde manier gestandaardiseerd. Men doet dus net hetzelfde met de testdata: men gebruikt er ook het gemiddelde en de standaardafwijking van de **trainingdata**. 

In [ ]:
# standaardiseren

# gemiddelde en standaardawijking van trainingdata bepalen
x_train_gem = np.mean(x_train)
x_train_std = np.std(x_train)
y_train_gem = np.mean(y_train)
y_train_std = np.std(y_train)

# trainingdata standaardiseren
x_train = (x_train - x_train_gem) / x_train_std
y_train = (y_train - y_train_gem) / y_train_std

# testdata standaardiseren
x_test = (x_test - x_train_gem) / x_train_std
y_test = (y_test - y_train_gem) / y_train_std

<div>
    <font color=#690027 markdown="1">   
        <h3>4.3 Regressielijn</h3> 
    </font>
</div>

De functies die nodig zijn om de regressielijn te bepalen, zijn nog steeds dezelfde.

In [ ]:
# regressielijn is rechte

# ingeven hoe vergelijking rechte is opgebouwd 
def rechte(x, a, b):
    """Voorschrift (schuine) rechte met variabele x en coëfficiënten a en b."""
    return a * x + b

# rechte zoeken die beste past bij bepaalde data, vergelijking tonen en coëfficiënten teruggeven
def linreg(x, y):
    """Rechte best passend bij data x en y."""
    popt, pcov = curve_fit(rechte, x, y)            # curve_fit() kijkt in def rechte() hoe functievoorschrift eruitziet
    # curve_fit() geeft twee zaken terug, waaraan gerefereerd wordt met popt en pcov
    # enkel eerste nodig, popt, die a en b van gezochte rechte geeft
    a, b = popt                                     # coëfficiënten
    print("y = ", a, "x +", b)                      # toon vergelijking regressielijn
    return a, b                                     # geeft coëfficiënten terug van vergelijking regressielijn

De regressielijn wordt echter enkel *gefit* op de trainingdata. Nadien wordt gekeken hoe goed deze regressielijn past bij de testdata.

In [ ]:
# lineaire regressie
# regressielijn bepalen a.d.h.v. trainingdata
a, b = linreg(x_train, y_train)
y_regressie = rechte(x_train, a, b)     

# gemiddelde kwadratische afwijking t.o.v. de trainingdata
print("Gemiddelde kwadratische afwijking voor de rechte m.b.t. de trainingdata: %.2f"% mean_squared_error(y_train, y_regressie))

In [ ]:
# regressielijn t.o.v. testdata
y_voorspeld = rechte(x_test, a, b)    

# gemiddelde kwadratische afwijking t.o.v. de testdata (generalisatie)
print("Gemiddelde kwadratische afwijking voor de rechte m.b.t. de testdata: %.2f"% mean_squared_error(y_test, y_voorspeld))

<div>
    <font color=#690027 markdown="1">   
        <h3>4.4 Grafiek</h3> 
    </font>
</div>

In [ ]:
# bereik inschatten
x_train.min(), x_train.max(), x_test.min(), x_test.max(), y_train.min(), y_train.max(), y_test.min(), y_test.max()

In [ ]:
# grafische voorstelling
plt.figure(figsize=(10, 8))

plt.xlim(x_train.min()-2, x_train.max()+2)
plt.ylim(y_train.min()-2, y_test.max()+2)
plt.title("Amazonewoud")
plt.xlabel("hoogte boom (gestandaardiseerd)")
plt.ylabel("stomatale dichtheid in mm² (gestandaardiseerd)")

plt.scatter(x_train, y_train, color="green", marker="o")
plt.plot(x_train, y_regressie, color="red")

# testdata
plt.scatter(x_test, y_test, color="blue", marker="o")

plt.show()

Interpretatie: 
Gemiddelde kwadratische afwijking voor de rechte m.b.t. de trainingdata is 0,93.
Gemiddelde kwadratische afwijking voor de rechte m.b.t. de testdata is 2,78. Deze fout is groter, dus niet zo'n goede generalisatie.<br>

In [ ]:
# vergelijking van rechte
print("De vergelijking van de rechte: y =", a, "x +",b)

In [ ]:
# vergelijking van rechte zonder standaardiseren
print("De vergelijking van de rechte: y =", 
      a * y_train_std/x_train_std, "x +",
      b * y_train_std + y_train_gem - a * x_train_gem *y_train_std/x_train_std)

<div>
    <font color=#690027 markdown="1">   
        <h2>5. Uitschieter</h2> 
    </font>
</div>

Merk op dat een bepaald punt van de testset allicht beschouwd kan worden als uitschieter. Bekijk eens wat de generalisatie is zonder dit punt. 

<div class="alert alert-block alert-warning">
In de notebooks 'Zeeniveau - Lineaire Regressie', 'Zeeniveau in Oostende - Regressie' en 'Zeeniveau in Oostende - ML Regressie' leer je hoe je naast een rechte een kromme vindt die het beste past bij een gegeven puntenwolk. Je leert er ook over underfitting en overfitting.   
</div>

<div>
    <h2>Referentielijst</h2> 
</div>

[1] Camargo, Miguel Angelo Branco, & Marenco, Ricardo Antonio. (2011). <br> &nbsp; &nbsp; &nbsp; &nbsp;Density, size and distribution of stomata in 35 rainforest tree species in Central Amazonia. Acta Amazonica, 41(2), 205-212. <br> &nbsp; &nbsp; &nbsp; &nbsp;https://dx.doi.org/10.1590/S0044-59672011000200004 en via e-mail.<br>

<div>
    <h2>Met steun van</h2> 
</div>

<img src="images/kikssteun.png" alt="Banner" width="1100"/>

<img src="images/cclic.png" alt="Banner" align="left" width="100"/><br><br>
Notebook KIKS, zie <a href="http://www.aiopschool.be">AI Op School</a>, van F. wyffels & N. Gesquière, is in licentie gegeven volgens een <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Naamsvermelding-NietCommercieel-GelijkDelen 4.0 Internationaal-licentie</a>.